In [3]:
# load the demographic data
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
df = pd.read_csv("demo-math.csv")
df.drop(df.columns[0], axis=1, inplace=True)
df.columns



Index(['dbn', 'school_name', 'year', 'total_enrollment',
       'grade_3k_pk_half_day_full', 'grade_k', 'grade_1', 'grade_2', 'grade_3',
       'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8', 'grade_9',
       'grade_10', 'grade_11', 'grade_12', 'female', 'female_1', 'male',
       'male_1', 'asian', 'asian_1', 'black', 'black_1', 'hispanic',
       'hispanic_1', 'multi_racial', 'multi_racial_1', 'native_american',
       'native_american_1', 'white', 'white_1', 'missing_race_ethnicity_data',
       'missing_race_ethnicity_data_1', 'students_with_disabilities',
       'students_with_disabilities_1', 'english_language_learners',
       'english_language_learners_1', 'poverty', 'poverty_1',
       'economic_need_index', 'district', 'boro', 'grade', 'category',
       'number_tested', 'mean_scale_score', 'level1_n', 'level1_', 'level2_n',
       'level2_', 'level3_n', 'level3_', 'level4_n', 'level4_', 'level3_4_n',
       'level3_4_'],
      dtype='object')

In [4]:
# display(df[["black_1", "mean_scale_score"]])

pct_black = df["black_1"]
math_avg = df["mean_scale_score"]

# np.corrcoef(pct_black, math_avg)

black_R = pearsonr(pct_black, math_avg)
pov_R = pearsonr(df["poverty_1"], math_avg)
print(black_R, pov_R)

(-0.47137621471716734, 0.0) (-0.6563811932689171, 0.0)


In [18]:
pearsonr(df["white_1"], df["mean_scale_score"])

(0.584611886499248, 0.0)

In [ ]:
np.corrcoef(pct_black, math_avg)


In [ ]:
from statsmodels.multivariate.manova import MANOVA


from statsmodels import multivariate as mv
import pingouin as pg

In [24]:
pg.corr(df["white_1"], df["mean_scale_score"])


# len(df["black_1"].unique())

pg.ancova(data=df, dv='mean_scale_score', covar='poverty_1', between='black_1')

,Source,SS,DF,F,p-unc,np2
0,black_1,1.019744e+06,718,7.491231,0.0,0.383161
1,poverty_1,6.591402e+05,1,3476.675117,0.0,0.286484
2,Residual,1.641653e+06,8659,NaN,NaN,NaN


In [9]:
fit = MANOVA.from_formula('black_1 + poverty_1 ~ mean_scale_score', data=df)

print(fit.mv_test())

                   Multivariate linear model
                                                                
----------------------------------------------------------------
       Intercept        Value  Num DF   Den DF   F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda 0.3690 2.0000 9376.0000 8017.4085 0.0000
         Pillai's trace 0.6310 2.0000 9376.0000 8017.4085 0.0000
 Hotelling-Lawley trace 1.7102 2.0000 9376.0000 8017.4085 0.0000
    Roy's greatest root 1.7102 2.0000 9376.0000 8017.4085 0.0000
----------------------------------------------------------------
                                                                
----------------------------------------------------------------
    mean_scale_score    Value  Num DF   Den DF   F Value  Pr > F
----------------------------------------------------------------
          Wilks' lambda 0.5062 2.0000 9376.0000 4572.6725 0.0000
         Pillai's trace 0.4938 2.0000 9376.00